In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

# Define LeNet-5 model without dropout for color images
class SimpleLeNet5Color(nn.Module):
    def __init__(self):
        super(SimpleLeNet5Color, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6 * 14 * 14, 10)  # Adjust input size based on your image dimensions

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 6 * 14 * 14)  # Adjust the view size based on your image dimensions
        x = self.fc1(x)
        return x

# Custom hook to register activations for conv1 layer only
class Conv1ActivationHook:
    def __init__(self):
        self.activations = []

    def __call__(self, module, input, output):
        if module == model.conv1:
            self.activations.append(output.cpu().detach().numpy())

# Load CIFAR-10 dataset for color images
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Loss function
criterion_color = nn.CrossEntropyLoss()

all_activation_patterns_list = []

# Train the model for color images
for training_iteration in range(10):  # Train the model 10 times

    # Initialize LeNet-5 model for color images outside the training loop
    model = SimpleLeNet5Color()

    optimizer_color = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

    # Register hook for activation patterns for conv1 layer only
    activation_hook = Conv1ActivationHook()
    hook_handle = model.register_forward_hook(activation_hook)

    # Lists to store training and testing accuracies for each epoch
    training_accuracies = []
    testing_accuracies = []

    for epoch in range(10):  # Change the number of epochs as needed
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer_color.zero_grad()
            outputs = model(inputs)
            loss = criterion_color(outputs, labels)
            loss.backward()
            optimizer_color.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = correct_train / total_train
        training_accuracies.append(train_accuracy)

        train_loss = running_loss / len(trainloader)
        print(f'Training Iteration {training_iteration + 1}, Epoch {epoch + 1}/{10}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

        # Test the model for color images
        correct_test = 0
        total_test = 0
        test_loss = 0.0

        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                outputs = model(inputs)
                loss = criterion_color(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_accuracy = correct_test / total_test
        testing_accuracies.append(test_accuracy)

        test_loss /= len(testloader)
        print(f'Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}')

    # Save activation patterns for conv1 layer for each run
    #activation_filename = f'conv1_activations2_{training_iteration}_{epoch}.npy'
    #np.save(activation_filename, np.array(activation_hook.activations))

    # Save the model for each run
    model_filename = f'model2_{training_iteration}_{epoch}.pth'
    torch.save(model.state_dict(), model_filename)

    # Save training and testing activities for each run
    training_filename = f'training_activities2_{training_iteration}_{epoch}.npy'
    testing_filename = f'testing_activities2_{training_iteration}_{epoch}.npy'

    np.save(training_filename, np.array(training_accuracies))
    np.save(testing_filename, np.array(testing_accuracies))

    # Remove the hook after each run
    hook_handle.remove()


Files already downloaded and verified
Files already downloaded and verified


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


Training Iteration 1, Epoch 1/10, Training Loss: 1.5313, Training Accuracy: 0.4588
Testing Loss: 1.3887, Testing Accuracy: 0.5166
Training Iteration 1, Epoch 2/10, Training Loss: 1.3663, Training Accuracy: 0.5237
Testing Loss: 1.3124, Testing Accuracy: 0.5451
Training Iteration 1, Epoch 3/10, Training Loss: 1.3156, Training Accuracy: 0.5431
Testing Loss: 1.3311, Testing Accuracy: 0.5394
Training Iteration 1, Epoch 4/10, Training Loss: 1.2885, Training Accuracy: 0.5538
Testing Loss: 1.2659, Testing Accuracy: 0.5635
Training Iteration 1, Epoch 5/10, Training Loss: 1.2673, Training Accuracy: 0.5615
Testing Loss: 1.3333, Testing Accuracy: 0.5389
Training Iteration 1, Epoch 6/10, Training Loss: 1.2561, Training Accuracy: 0.5665
Testing Loss: 1.2904, Testing Accuracy: 0.5606
Training Iteration 1, Epoch 7/10, Training Loss: 1.2448, Training Accuracy: 0.5713
Testing Loss: 1.2865, Testing Accuracy: 0.5611
Training Iteration 1, Epoch 8/10, Training Loss: 1.2374, Training Accuracy: 0.5747
Testing